APIキーを定義、パッケージimport、関数を定義します。

In [1]:
%env OPENAI_API_KEY=

env: OPENAI_API_KEY=


In [11]:
%pip install ntplib

Note: you may need to restart the kernel to use updated packages.


[chat/completions API](https://platform.openai.com/docs/api-reference/chat/create) を呼び出す関数を定義します。

In [3]:
import json
import requests
import os

GPT_MODEL = "gpt-3.5-turbo-0613"

# Open AI API呼び出し関数
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.environ["OPENAI_API_KEY"],
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


今回function callingのテストに使用する関数を定義します。引数にタイムゾーンを与え、現在の日時を取得する簡単な関数を用います。

In [4]:
import ntplib
import pytz
from datetime import datetime, timezone

# 今回function callingで実行する関数
def fetch_current_datetime_in_timezone(timezone_str):
    client = ntplib.NTPClient()
    response = client.request('pool.ntp.org')
    utc_now = datetime.fromtimestamp(response.tx_time, timezone.utc)
    try:
        target_timezone = pytz.timezone(timezone_str)
        return utc_now.astimezone(target_timezone).strftime('%Y-%m-%d %H:%M:%S.%f %Z%z')
    except pytz.UnknownTimeZoneError:
        raise ValueError(f"Unsupported timezone: {timezone_str}")

# Open AIに通知するfunctionsパラメータ
functions = [
    {
        "name": "fetch_current_datetime_in_timezone",
        "description": "Enter time zone (e.g., Asia/Tokyo, UTC, America/New_York, Europe/London, etc.). Outputs the current date and time corresponding to the time zone.",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone_str": {
                    "type": "string",
                    "description": "Time zone (e.g., Asia/Tokyo, UTC, America/New_York, Europe/London, etc.). Infer this from the conversation.",
                }
            },
            "required": ["timezone_str"],
        },
    }
]


"What time is it now in tokyo, Japan?" という質問とfunctions JsonパラメータをOpen AI API `chat/completions` にPOSTします。

In [5]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What time is it now in tokyo, Japan?"})
chat_response = chat_completion_request(
    messages, functions=functions
)


Open AI API `chat/completions` は以下の通り結果を返却します。Open AI API側で、"What time is it now in tokyo, Japan?"という文章から、タイムゾーンとして"Asia/Tokyo"を抜き出していることがわかります。

In [6]:
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'fetch_current_datetime_in_timezone',
  'arguments': '{\n  "timezone_str": "Asia/Tokyo"\n}'}}

得られた引数情報を使って、関数をUser側で実行します。以下の通り、関数を実行することで現在時刻を出力できました。

In [7]:
if assistant_message.get("function_call"):
    function_call_result = assistant_message["function_call"]
    
    # Get function by name from global scope
    func = globals().get(function_call_result["name"])
    if not func:
        raise ValueError(f"No function named '{function_call_result['name']}' found.")
    
    # Parse arguments and call the function
    arguments = json.loads(function_call_result["arguments"])
    response = func(**arguments)
    print("Answer: ", response)    

Answer:  2023-09-11 11:31:59.513282 JST+0900


関数の実行結果を含めたこれまでの会話をOpen AI API `chat/completions` にPOSTします。

In [8]:
messages.append({"role": "function", "name": "fetch_current_datetime_in_timezone", "content": response})
print(messages)
chat_response = chat_completion_request(
    messages, functions=functions
)

[{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}, {'role': 'user', 'content': 'What time is it now in tokyo, Japan?'}, {'role': 'assistant', 'content': None, 'function_call': {'name': 'fetch_current_datetime_in_timezone', 'arguments': '{\n  "timezone_str": "Asia/Tokyo"\n}'}}, {'role': 'function', 'name': 'fetch_current_datetime_in_timezone', 'content': '2023-09-11 11:31:59.513282 JST+0900'}]


関数の実行結果と会話から尤もらしい回答をOpen AI APIが返却してくれます。

In [9]:
assistant_message = chat_response.json()["choices"][0]["message"]
messages.append(assistant_message)
print(assistant_message)

{'role': 'assistant', 'content': 'The current time in Tokyo, Japan is 11:31 AM on September 11, 2023.'}
